In [1]:
import ee
import geemap
import numpy as np
import json
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import geemap.colormaps as geecm

In [2]:
# # # Trigger the authentication flow.
# ee.Authenticate()

# # # Initialize the library.
ee.Initialize()

In [3]:
# define folder and file paths
folder_HMA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"HMA2")
regions_fp = os.path.join(folder_HMA, 'Datasets', 'AOI definition', 'Final AOI', "Final Products", 'subregion_boundaries.shp')
rgi_5km_fp = os.path.join(folder_HMA, 'Datasets', 'AOI definition', 'Final AOI', "Final Products", 'AOI_5km.shp')
fp_dc = os.path.join(folder_HMA, 'Datasets', 'AOI definition', 'Final AOI', "Final Products", 'AOI_dc_rgi.shp')

In [7]:
# define bounding box geometry of our study area
geom = ee.Geometry.BBox(60, 20, 110, 45)

In [13]:
# load the climate data, select variables and dates
climate_data = (ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
                    .filterDate('1950-01-01', '2024-01-01')
                    .select(['temperature_2m', 'total_precipitation_sum'])
               )

# 'crs': 'EPSG:4326', 'transform': [0.1, 0, -180.05, 0, -0.1, 90.05]

In [25]:
# define annual periods to group together. seasons.
year_bins = [ [1950,1979], [1980,1989], [1990,1999], [2000,2009], [2010,2019], [2014,2023] ]
month_bins = [ [12,2, 'winter'],
           [3,5,  'spring'],
           [6,8,  'summer'],
           [9,11, 'fall'],
           [1,12, 'annual'] ]

In [35]:
# compute winter variables
c=0
for ys in year_bins:
    for ms in month_bins:
        
        # count number of months included
        if ms[2]=='winter': n_months = (3)*(ys[1]-ys[0])
        else: n_months = (ms[1]-ms[0]+1)*(ys[1]-ys[0])
        
        # load average temperature for this season/years
        temp = climate_data.select('temperature_2m') \
                .filter(ee.Filter.calendarRange(ys[0],ys[1],'year')) \
                .filter(ee.Filter.calendarRange(ms[0],ms[1],'month')) \
                .mean()#.clip(geom)

        # load total precip per year for this season/years
        precip = climate_data.select('total_precipitation_sum') \
                .filter(ee.Filter.calendarRange(ys[0],ys[1],'year')) \
                .filter(ee.Filter.calendarRange(ms[0],ms[1],'month')) \
                .sum().divide(n_months)#.clip(geom)

        # save to local drive, temp first, then precip
        out_name = f"t_{ms[2]}_{ys[0]}_{ys[1]}.tif"
        out_path = os.path.join(folder_HMA, 'Datasets', 'climate', 'era5land_tifs', out_name)
        print(f"\n{out_name}")
        geemap.ee_export_image( temp, filename=out_path, unmask_value=0,
                               region=geom, crs='EPSG:4326', crs_transform=[0.1, 0, -180.05, 0, -0.1, 90.05])
        
        out_name = f"p_{ms[2]}_{ys[0]}_{ys[1]}.tif"
        out_path = os.path.join(folder_HMA, 'Datasets', 'climate', 'era5land_tifs', out_name)
        print(f"\n{out_name}")
        geemap.ee_export_image( precip, filename=out_path, unmask_value=0,
                               region=geom, crs='EPSG:4326', crs_transform=[0.1, 0, -180.05, 0, -0.1, 90.05])
        
print("Done!")


t_winter_1950_1979.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\lzell\OneDrive - Colostate\Desktop\HMA2\Datasets\climate\era5land_tifs\t_winter_1950_1979.tif

p_winter_1950_1979.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\lzell\OneDrive - Colostate\Desktop\HMA2\Datasets\climate\era5land_tifs\p_winter_1950_1979.tif

t_spring_1950_1979.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\lzell\OneDrive - Colostate\Desktop\HMA2\Datasets\climate\era5land_tifs\t_spring_1950_1979.tif

p_spring_1950_1979.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\lzell\OneDrive - Colostate\Desktop\HMA2\Datasets\climate\era5land_tifs\p_spring_1950_1979.tif

t_summer_1950_1979.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\lzell\OneDrive - Colostate\Desktop\HMA2\Datasets\climate\era5land_tifs\t_summer_1950_1979.tif

p_summer_1950_1979.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\lzell\On